In [144]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

import os
from pprint import pprint

#using this to prevent a site from kicking us out because it acts more like a human and less like an automated process
import time

import re

## * Get Access *

In [278]:
#attempting to access the website - blocked because it sees a machine and not a human
url = 'https://codeup.com/blog/'
response = get(url)
response

<Response [403]>

In [279]:
#check the header because some sites do not allow python-requests
requests.utils.default_headers()

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [280]:
#assign new user_agent
user_agent = ["Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"]

In [281]:
#changing the header from python to Chrome
headers = {"User-Agent": "Chrome/91.0.4472.124"}
response = get(url, headers=headers)
response

<Response [200]>

## 1. 5 distinct blog posts. For each post, you should scrape at least the post's title and content.
* Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article.

In [282]:
#looking at the html script to ensure it pulled
pprint(response.text[:400])

('<!DOCTYPE html>\n'
 '<html lang="en-US">\n'
 '<head>\n'
 '\t<meta charset="UTF-8" />\n'
 '<meta http-equiv="X-UA-Compatible" content="IE=edge">\n'
 '\t<link rel="pingback" href="https://codeup.com/xmlrpc.php" />\n'
 '\n'
 '\t<script type="text/javascript">\n'
 "\t\tdocument.documentElement.className = 'js';\n"
 '\t</script>\n'
 '\t\n'
 '\t<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin '
 '/><script id="diviarea-loader">window.DiviPopupData=wi')


In [283]:
#make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://codeup.com/xmlrpc.php" rel="pingback"/>
<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><script id="diviarea-loader">window.DiviPopupData=window.DiviAreaConfig={"zIndex":1000000,"animateSpeed":400,"triggerClassPrefix":"show-popup-","idAttrib":"data-popup","modalIndicatorClass":"is-modal","blockingIndicatorClass":"is-blocking","defaultShowCloseButton":true,"withCloseClass":"with-close","noCloseClass":"no-close","triggerCloseClass":"close","singletonClass":"single","darkModeClass":"dark","noShadowClass":"no-shadow","altCloseClass":"close-alt","popupSelector":".et_pb_section.popup","initializeOnEvent":"et_pb_after_init_modules","popupWrapperClass":"area-outer-wrap","fullHeightClass":"full-height","openPopupClass":"da-overlay-visible","ove

In [284]:
#gave the html script for the webpage title element
soup.h1

<h1>Codeup Blog</h1>

In [285]:
#after inspecting the website - scrub for all h2
soup.find_all('h2')

[<h2 class="entry-title"><a href="https://codeup.com/featured/apida-heritage-month/">Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-panelist-spotlight/">Women in tech: Panelist Spotlight – Magdalena Rahn</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-rachel-robbins-mayhill/">Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/">Women in Tech: Panelist Spotlight – Sarah Mellor</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/events/women-in-tech-madeleine/">Women in Tech: Panelist Spotlight – Madeleine Capper</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/panelist-spotlight-4/">Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia</a></h2>

In [286]:
#after inspecting the website - scrub for all anchors to get the url for the blogposts
entry_url = soup.find_all('a', class_='entry-featured-image-url')
entry_url[0]['href']

'https://codeup.com/featured/apida-heritage-month/'

In [287]:
#pulls back all the headers and searched for the class_ name pulled from the inspection of the website for entry-title
article_links = soup.find_all('h2', class_='entry-title')
article_links

[<h2 class="entry-title"><a href="https://codeup.com/featured/apida-heritage-month/">Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-panelist-spotlight/">Women in tech: Panelist Spotlight – Magdalena Rahn</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/featured/women-in-tech-rachel-robbins-mayhill/">Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/">Women in Tech: Panelist Spotlight – Sarah Mellor</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/events/women-in-tech-madeleine/">Women in Tech: Panelist Spotlight – Madeleine Capper</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/codeup-news/panelist-spotlight-4/">Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia</a></h2>

In [288]:
#parse 1st link only and pull back title
article_links[0].text

'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa'

In [289]:
#for loop to iterate and print the title of each element
title_list=[]

#for loop to iterate and print content of each element
num_elementst = len(article_links)


#using np.arange to iterate through soup skipping every other line (it was a date, didn't need it)
for i in np.arange(num_elementst):
    title_list.append(article_links[i].text)
title_list

['Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
 'Women in tech: Panelist Spotlight – Magdalena Rahn',
 'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill',
 'Women in Tech: Panelist Spotlight – Sarah Mellor',
 'Women in Tech: Panelist Spotlight – Madeleine Capper',
 'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia']

In [290]:
#content locations are 1, 3, 5, 7, 9, 11
soup.find_all('p')[1].text

'May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...'

In [291]:
#empty list
content_list=[]

#for loop to iterate and print content of each element
num_elements = len(soup.find_all('p'))

#using np.arange to iterate through soup skipping every other line (it was a date, didn't need it)
for i in np.arange(num_elements - 12, step=2):
    content_list.append(soup.find_all('p')[i+1].text)
content_list

['May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...',
 'Women in tech: Panelist Spotlight – Magdalena Rahn Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
 'Women in tech: Panelist Spotlight - Rachel Robbins-Mayhill Codeup is hosting a Women in Tech Panel in honor of Women’s...',
 'Women in tech: Panelist Spotlight - Sarah Mellor\xa0 Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
 'Women in tech: Panelist Spotlight - Madeleine Capper Codeup is hosting a Women in Tech Panel in honor of Women’s...',
 'Black excellence in tech: Panelist Spotlight - Wilmarie De La Cruz Mejia Codeup is hosting a Black Excellence in Tech...']

In [292]:
#grabbing the date becuase why not
date_list=[]

#for loop to iterate and print content of each element
num_elementsd = len(soup.find_all('p'))

for i in np.arange(num_elementsd - 12, step=2):
    date_list.append(soup.find_all('p')[i].text)
date_list

['May 24, 2023',
 'Mar 28, 2023',
 'Mar 20, 2023',
 'Mar 13, 2023',
 'Mar 6, 2023',
 'Feb 16, 2023']

In [293]:
# format for the dictionary
# {
#     'title': 'the title of the article',
#     'content': 'the full text content of the article',
#     'date' : 'added for extra practice'
# }


In [294]:
#combine lists to one dictionary following assigned format
results = {'title':title_list
          , 'content':content_list
          , 'date' : date_list}
results

{'title': ['Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa',
  'Women in tech: Panelist Spotlight – Magdalena Rahn',
  'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill',
  'Women in Tech: Panelist Spotlight – Sarah Mellor',
  'Women in Tech: Panelist Spotlight – Madeleine Capper',
  'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia'],
 'content': ['May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the...',
  'Women in tech: Panelist Spotlight – Magdalena Rahn Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
  'Women in tech: Panelist Spotlight - Rachel Robbins-Mayhill Codeup is hosting a Women in Tech Panel in honor of Women’s...',
  'Women in tech: Panelist Spotlight - Sarah Mellor\xa0 Codeup is hosting a Women in Tech Panel in honor of Women’s History...',
  'Women in tech: Panelist Spotlight - Madeleine Capper Codeup is hos

In [295]:
#turn to df
results_df = pd.DataFrame(results)
results_df

,title,content,date
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,"May 24, 2023"
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,"Mar 28, 2023"
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight - Rachel Rob...,"Mar 20, 2023"
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight - Sarah Mell...,"Mar 13, 2023"
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight - Madeleine ...,"Mar 6, 2023"
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight -...,"Feb 16, 2023"


In [296]:
def get_blog_articles(url):
    '''
    this function grabs title, content, and date from Codeup Blog site using new user_agent that is not a python request
    '''
    #assign new user_agent
    user_agent = ["Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"]
    
    #changing the header from python to Chrome
    headers = {"User-Agent": "Chrome/91.0.4472.124"}
    response = get(url, headers=headers)
    
    #make a soup variable holding the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #pulls back all the headers and searched for the class_ name pulled from the inspection of the website for entry-title
    article_links = soup.find_all('h2', class_='entry-title')

    #empty lists
    title_list=[]
    content_list=[]
    date_list = []
    
    #for loop to iterate and print content of each element
    num_elementst = len(article_links)
    num_elements = len(soup.find_all('p'))
    num_elementsd = len(soup.find_all('p'))

    #using np.arange to iterate through soup skipping every other line
    for i in np.arange(num_elementst):
        title_list.append(article_links[i].text)
    for i in np.arange(num_elements - 12, step=2):
        content_list.append(soup.find_all('p')[i+1].text)
    for i in np.arange(num_elementsd - 12, step=2):
        date_list.append(soup.find_all('p')[i].text)
        
    #combine lists to one dictionary following assigned format
    results = {'title':title_list
              ,'content':content_list
              ,'date' : date_list}
    
    #turn to df
    results_df = pd.DataFrame(results)
    
    return results_df


In [297]:
results_df = get_blog_articles(url)
results_df

,title,content,date
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,"May 24, 2023"
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,"Mar 28, 2023"
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight - Rachel Rob...,"Mar 20, 2023"
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight - Sarah Mell...,"Mar 13, 2023"
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight - Madeleine ...,"Mar 6, 2023"
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight -...,"Feb 16, 2023"


## * Get Access *

In [451]:
#attempting to access the website - not blocked, accepts python requests
url = 'https://inshorts.com/en/read'
response = get(url)
response

<Response [200]>

## 2. We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.
* Write a function called get_news_articles that scrapes the news articles for the following topics:
    * Business
    * Sports
    * Technology
    * Entertainment

In [452]:
pprint(response.text[:400])

('<!doctype html>\n'
 '<html lang="en">\n'
 '\n'
 '<head>\n'
 '  <meta charset="utf-8" />\n'
 '  <style>\n'
 '    /* The Modal (background) */\n'
 '    .modal_contact {\n'
 '        display: none; /* Hidden by default */\n'
 '        position: fixed; /* Stay in place */\n'
 '        z-index: 8; /* Sit on top */\n'
 '        left: 0;\n'
 '        top: 0;\n'
 '        width: 100%; /* Full width */\n'
 '        height: 100%;\n'
 '        overflow: auto; /* Enable scroll if ne')


In [453]:
# url = 'https://inshorts.com/en/business'
# response = get(url)
# response

In [454]:
#make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positi

In [311]:
#after inspecting the website - scrub for all li to find Business, Sports, Technology, Entertainment...
soup.find_all('li')

[<h1>Stay informed in 60 words.</h1>]

In [333]:
category_links = soup.find_all('a')
category_links

[<a href="/"><img class="logo" src="https://assets.inshorts.com/website_assets/images/logo_inshorts.png"/></a>,
 <a href="/en/read"><div class="lang-en selected">English</div></a>,
 <a href="/hi/read"><div class="lang-hi">हिन्दी</div></a>,
 <a href="/en/read" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToAllNews', 'action': 'clicked', 'label': 'RedirectedToAllNews' });"> <li class="active-category selected">All News</li> </a>,
 <a href="/en/read/national" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToIndiaNews', 'action': 'clicked', 'label': 'RedirectedToIndiaNews' });"> <li class="active-category">India</li> </a>,
 <a href="/en/read/business" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkToBusinessNews', 'action': 'clicked', 'label':  'RedirectedToBusinessNews' });"> <li class="active-category">Business</li> </a>,
 <a href="/en/read/sports" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'LinkTosportsNews', 'acti

In [359]:
# #make url_list to cycle through the sites I want because it was too complicated the other way
# url_list = ['/business', '/sports', '/technology', '/entertainment']

In [429]:
# #cycle through list with timer to allow for load time printing response to ensure I have access
# url = 'https://inshorts.com/en/read'
# #create a dictionary to store soup variables for each endpoint
# soup_dict_ = {}
# response_list=[]
# for endpoint in url_list:
#     response = get(url)
#     print(f'{url}{endpoint}')
#     time.sleep(5)
#     print(response)
#     response_list.append(response)
#     print(f'access granted')

#     #iterate through the response_list and create a soup variable for each response
#     for endpoint, response in zip(url_list, response_list):
#         soup = BeautifulSoup(response.content, 'html.parser')
#         soup_dict_[endpoint.replace('/','')] = soup
#     print(f"soup_dict_{endpoint.replace('/','')}")
#     # Now you have a dictionary with keys as endpoints and values as the corresponding soup variables
#     print(f'soup made successfully')

https://inshorts.com/en/read/business
<Response [200]>
access granted
soup_dict_business
soup made successfully
https://inshorts.com/en/read/sports
<Response [200]>
access granted
soup_dict_sports
soup made successfully
https://inshorts.com/en/read/technology
<Response [200]>
access granted
soup_dict_technology
soup made successfully
https://inshorts.com/en/read/entertainment
<Response [200]>
access granted
soup_dict_entertainment
soup made successfully


In [439]:
#dictionary made encasing all soups
soup_dict_.keys()

dict_keys(['business', 'sports', 'technology', 'entertainment'])

In [440]:
#i still need to figure out what the class titles are called...

In [486]:
soup.find_all(class_='news-card-title')

[<div class="news-card-title news-right-box">
 <a class="clickable" href="/en/news/virat-kohli-completes-12-years-in-test-cricket-says-forever-grateful-1687282886037" onclick="track_GA_Mixpanel({'hitType': 'event', 'category': 'TitleOfNews', 'action': 'clicked', 'label': 'Virat%20Kohli%20completes%2012%20years%20in%20Test%20cricket%2C%20says%20'Forever%20grateful')' });" style="color:#44444d!important">
 <span itemprop="headline">Virat Kohli completes 12 years in Test cricket, says 'Forever grateful'</span>
 </a>
 <div class="news-card-author-time news-card-author-time-in-title">
 <a href="/prev/en/news/virat-kohli-completes-12-years-in-test-cricket-says-forever-grateful-1687282886037"><span class="short">short</span></a> by <span class="author">Anmol Sharma</span> / 
       <span class="time" content="2023-06-20T17:41:26.000Z" itemprop="datePublished">11:11 pm</span> on <span clas="date">20 Jun 2023,Tuesday</span>
 </div>
 </div>,
 <div class="news-card-title news-right-box">
 <a clas

In [540]:
# url = 'https://inshorts.com/en/read/technology'
# response = get(url)
# soup = BeautifulSoup(response.content, 'html.parser')
# soup.find_all(itemprop='headline')


# #title list for titles
# title_list = []

# #create variable
# titles = soup.find_all(itemprop='headline')

# #establishing range for loop
# num_titles = len(soup.find_all(itemprop='headline'))

# #using np.arange to iterate through 
# for i in np.arange(num_titles):
#     title_list.append(titles[i].text)
# title_list

[<span itemprop="headline">WhatsApp lets users silence calls from unknown numbers amid rise in spam calls</span>,
 <span itemprop="headline">Suzuki to start making 'flying cars' with SkyDrive by 2024 spring</span>,
 <span itemprop="headline">Apple could force 111-year-old Swiss firm to change its apple logo</span>,
 <span itemprop="headline">Realme disables feature amid claims it collected SMS, call data</span>,
 <span itemprop="headline">How can you silence calls from unknown numbers on WhatsApp?</span>,
 <span itemprop="headline">What is the Pink WhatsApp scam that authorities have warned people about?</span>,
 <span itemprop="headline">Google plans to assemble Pixel phones in India: Report</span>,
 <span itemprop="headline">Reddit flooded with British comedian's pics to protest API charges</span>,
 <span itemprop="headline">This AI breakthrough is redefining healthcare diagnosis!</span>,
 <span itemprop="headline">Apple Watch helped detect blood clot, saved my life: US woman</span>,

In [501]:
#now I need to figure out how to get the content

In [506]:
#found the contents
soup.find_all(itemprop='articleBody')

[<div itemprop="articleBody">Ex-Team India captain Virat Kohli on Tuesday completed 12 years in Test cricket. Sharing a picture of himself batting in the recent WTC 2023 final, Kohli wrote, "12 years in Test cricket today. Forever grateful." Kohli made his Test debut against West Indies in Kingston on June 20, 2011. Kohli has scored 8,479 runs in 185 Test innings so far.</div>,
 <div itemprop="articleBody">Australia chased down 281-run target against England in first Ashes 2023 Test at Edgbaston to record their highest successful chase in Ashes in nearly 75 years. Australia achieved the target with 4.3 overs remaining on the last day, driven by Usman Khawaja's 65(197) and captain Pat Cummins' 44*(73). Notably, England had declared their first innings at 393/8 on Day 1.</div>,
 <div itemprop="articleBody">The Kerala High Court recently observed that though a mother may be considered "morally bad in the societal sense", it doesn't indicate that she's bad for the child's welfare. "In a ma

In [504]:
#create variable
contents = soup.find_all(itemprop='articleBody')
contents

[<div itemprop="articleBody">Ex-Team India captain Virat Kohli on Tuesday completed 12 years in Test cricket. Sharing a picture of himself batting in the recent WTC 2023 final, Kohli wrote, "12 years in Test cricket today. Forever grateful." Kohli made his Test debut against West Indies in Kingston on June 20, 2011. Kohli has scored 8,479 runs in 185 Test innings so far.</div>,
 <div itemprop="articleBody">Australia chased down 281-run target against England in first Ashes 2023 Test at Edgbaston to record their highest successful chase in Ashes in nearly 75 years. Australia achieved the target with 4.3 overs remaining on the last day, driven by Usman Khawaja's 65(197) and captain Pat Cummins' 44*(73). Notably, England had declared their first innings at 393/8 on Day 1.</div>,
 <div itemprop="articleBody">The Kerala High Court recently observed that though a mother may be considered "morally bad in the societal sense", it doesn't indicate that she's bad for the child's welfare. "In a ma

In [508]:
#length of contents - verifying same shape
num_content = len(soup.find_all(itemprop='articleBody'))
num_content

25

In [518]:
#content list for contents
content_list = []

#create variable
contents = soup.find_all(itemprop='articleBody')

#establishing range for loop
num_content = len(soup.find_all(itemprop='articleBody'))

#using np.arange to iterate through 
for i in np.arange(num_content):
    content_list.append(contents[i].text)
content_list

['Ex-Team India captain Virat Kohli on Tuesday completed 12 years in Test cricket. Sharing a picture of himself batting in the recent WTC 2023 final, Kohli wrote, "12 years in Test cricket today. Forever grateful." Kohli made his Test debut against West Indies in Kingston on June 20, 2011. Kohli has scored 8,479 runs in 185 Test innings so far.',
 "Australia chased down 281-run target against England in first Ashes 2023 Test at Edgbaston to record their highest successful chase in Ashes in nearly 75 years. Australia achieved the target with 4.3 overs remaining on the last day, driven by Usman Khawaja's 65(197) and captain Pat Cummins' 44*(73). Notably, England had declared their first innings at 393/8 on Day 1.",
 'The Kerala High Court recently observed that though a mother may be considered "morally bad in the societal sense", it doesn\'t indicate that she\'s bad for the child\'s welfare. "In a matter related to the child\'s custody, the welfare aspect alone has to be considered," it

In [533]:
# #make the category section now - hard code it
# category_list=[]

In [572]:
# #hmm, this might work? try with business url
# b_url = url+'/business'
# b_response = get(url)
# soup = BeautifulSoup(b_response.content, 'html.parser')
# soup.find_all('category')

#did not work - abort using dictionary method

In [569]:
def get_news_articles():
    '''
    this function gets the top 25 news articles titles, content, and category from inshorts.com
    category is only business, sports, technology, and entertainment
    '''
    #create a dictionary and a list to store variables
    soup_dict_ = {} #stores soup calls for each of the endpoints in url_list
    response_list = [] #stores the responses to iterate
    title_list = []
    content_list = []
    category_list = []

    #make url_list to cycle through the sites I want because it was too complicated the other way
    url_list = ['/business', '/sports', '/technology', '/entertainment']

    #access the website - not blocked, accepts python requests
    url = 'https://inshorts.com/en/read'
    response = get(url)

    #cycle through list with timer to allow for load time printing response to ensure I have access
    for endpoint in url_list:
        response = get(url)
        print(f'{url}{endpoint}')
        time.sleep(5)
        print(response)
        response_list.append(response)
        print(f'access granted')

        #iterate through the response_list and create a soup variable for each response
        for endpoint, response in zip(url_list, response_list):
            soup = BeautifulSoup(response.content, 'html.parser')
            soup_dict_[endpoint.replace('/','')] = soup

            #create variables
            titles = soup.find_all(itemprop='headline')
            contents = soup.find_all(itemprop='articleBody')

            #establishing range for loop
            num_titles = len(soup.find_all(itemprop='headline'))
            num_content = len(soup.find_all(itemprop='articleBody'))
            num_category = len(soup.find_all(itemprop='articleBody'))

        #using np.arange to iterate through 
        for i in np.arange(num_titles):
            title_list.append(titles[i].text)
        for i in np.arange(num_content):
            content_list.append(contents[i].text)
        for i in np.arange(num_category):
            category_list.append(endpoint.replace('/',''))

        #combine lists to one dictionary following assigned format
        results = {'title':title_list,
                   'content':content_list,
                   'category':category_list}

    results_df = pd.DataFrame(results)
    return results_df

In [570]:
results_df = get_news_articles()
results_df

https://inshorts.com/en/read/business
<Response [200]>
access granted
https://inshorts.com/en/read/sports
<Response [200]>
access granted
https://inshorts.com/en/read/technology
<Response [200]>
access granted
https://inshorts.com/en/read/entertainment
<Response [200]>
access granted


,title,content,category
0,Australia record their highest successful run ...,Australia chased down 281-run target against E...,business
1,Virat Kohli completes 12 years in Test cricket...,Ex-Team India captain Virat Kohli on Tuesday c...,business
2,Man with 5 wives allegedly kidnaps another wom...,A man named Rashid has reportedly kidnapped a ...,business
3,Mother labelled 'morally bad' by society may b...,The Kerala High Court recently observed that t...,business
4,Worst form of inhumanity: Father of 7-yr-old M...,"The father of a seven-year-old boy, who was bu...",business
...,...,...,...
95,A natural calamity: UP minister on Ballia deat...,After over 60 people died in Ballia district h...,entertainment
96,Rajnath Singh to observe International Yoga Da...,Defence Minister Rajnath Singh will join India...,entertainment
97,Fire at e-bike shop spreads to apartments in N...,A fire that broke out at an e-bike shop in New...,entertainment
98,"NSE, NCL settle trade glitch case, execs order...",NSE and NSE Clearing (NCL) have settled a case...,entertainment
